In [2]:
import torch

In [2]:
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [3]:
print(x.grad)  # 默认不会保存梯度
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)

None


tensor([0., 1., 2., 3.], requires_grad=True)

In [4]:
y = 2 * torch.dot(x, x)  # y=2x^2
y

tensor(28., grad_fn=<MulBackward0>)

In [5]:
# 通过调⽤反向传播函数来⾃动计算y关于x每个分量的梯度，并打印这些梯度。
# y'=4x
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [6]:
# 验证x.grad是否等于4*x
4 * x == x.grad

tensor([True, True, True, True])

In [7]:
# 在默认情况下， PyTorch会累积梯度，我们需要清除之前的值，就是将grad都设置为0
x.grad.zero_()
print(x.grad)
# y=x1+x2+x3+x4
y = x.sum()
y.backward()
x.grad

tensor([0., 0., 0., 0.])


tensor([1., 1., 1., 1.])

In [9]:
"""
注意这里的区别，这时y已经不是标量了，y是一个向量
此时再计算梯度时，即但当调⽤向量的反向计算时，
我们通常会试图计算⼀批训练样本中每个组成部分的损失函数的导数。
这⾥，我们的⽬的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。
"""
x.grad.zero_()
# x*x注意和torch.dot(x, x)区别，torch.dot(x, x)=>x1^2+x2^2+..+x4^2
# x*x=> x1^2 | x2^2 | ... | x4^2，其中| 表示元素之间相互没关系
y = x * x
print(y)  # y是一个向量，而不是标量
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)


tensor([0., 2., 4., 6.])

In [14]:
# 分离计算
"""
如果z=u*x，而u=x^2，如果直接计算梯度的话，z'=3x^2
如果我们想让u视为一个常数，即z'=u,则需要丁勇detach
"""
# 不分离的情况
x.grad.zero_()
y = x * x
u = y
z = u * x
z.sum().backward()
print(x.grad)  # z'=3x^2
# 分离的情况
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x
z.sum().backward()
print(x.grad)  # z'=u,u视为一个常数，u=x^2
# 后面如果还想获得y'对应的梯度，即y'=2x
x.grad.zero_()
y.sum().backward()
print(x.grad)

tensor([ 0.,  3., 12., 27.])
tensor([0., 1., 4., 9.])
tensor([0., 2., 4., 6.])


In [14]:
def f(a):
    b = a * 2
    # norm求L2范数
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c
a = torch.randn(size=(), requires_grad=True)
print(a)
"""
d
"""
d=f(a)
""""
f(a)明显是非线性函数，根据f(a)的定义可以确定d=ka，k是一个常数
f‘(a)=k,d=ka
f'(a)==a.grad==d/a
"""
d.backward()
print(a.grad)
a.grad==d/a

tensor(1.1101, requires_grad=True)
tensor(1024.)


tensor(True)